# 07: PyTorch Experiment Tracking

This notebook tries several different models with different types of data augmentation and tracks the results to compare them.

## TODO:
* Setup different experiments
* Track different experiments with SummaryWriter/PyTorch profiler (can view these on Weights & Biases)
    * 1. EffNetB0 + no augmentation
    * 2. EffNetB0 + random augmentation
    * 3. EffNetB4 + no augmentation
    * 4. EffNetB4 + random augmentation
    * Mixed precision?

* See here for more on the profiler: https://pytorch.org/blog/introducing-pytorch-profiler-the-new-and-improved-performance-tool/
* See here on setting up TensorBoard in VSCode: https://code.visualstudio.com/docs/datascience/pytorch-support

Start with one model + tracking experiments, then upgrade to more.

In [2]:
import torch
import torchvision
from torch import nn
from torchvision import transforms, datasets
from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt

from going_modular import data_setup, engine

## Setup device

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Get and load data

In [11]:
import os
import requests
from zipfile import ZipFile

def get_food_image_data():
    if not os.path.exists("data/10_whole_foods"):
        os.makedirs("data/", exist_ok=True)
        # Download data
        data_url = "https://storage.googleapis.com/food-vision-image-playground/10_whole_foods.zip"
        print(f"Downloading data from {data_url}...")
        requests.get(data_url)
        # Unzip data
        targ_dir = "data/10_whole_foods"
        print(f"Extracting data to {targ_dir}...")
        with ZipFile("10_whole_foods.zip") as zip_ref:
            zip_ref.extractall(targ_dir)
    else:
        print("data/10_whole_foods dir exists, skipping download")

get_food_image_data()

Extracting data to data/10_whole_foods...


In [12]:
# Setup dirs
train_dir = "data/10_whole_foods/train"
test_dir = "data/10_whole_foods/test"

# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Create starter transform
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])           

# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f62f6cb4850>,
 ['apple',
  'banana',
  'beef',
  'blueberries',
  'carrots',
  'chicken_wings',
  'egg',
  'honey',
  'mushrooms',
  'strawberries'])

## Load model 

In [36]:
model = torchvision.models.efficientnet_b0(pretrained=True).to(device)
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [37]:
# Update the classifier
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(1280, len(class_names)).to(device))

# Freeze all base layers 
for param in model.features.parameters():
    param.requires_grad = False

## Train model and track results

In [26]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Adjust training function to track results with `SummaryWriter`

In [32]:
model.name = "EfficietNetB0"
model.name

'EfficietNetB0'

In [27]:
from torch.utils.tensorboard import SummaryWriter
from going_modular.engine import train_step, test_step
from tqdm import tqdm
writer = SummaryWriter()

In [28]:
def train(
    model,
    train_dataloader,
    test_dataloader,
    optimizer,
    loss_fn=nn.CrossEntropyLoss(),
    epochs=5,
):

    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
        )
        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn
        )

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Add results to SummaryWriter
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)
        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
    
    # Close the writer
    writer.close()

    return results

In [29]:
# Train model
# Note: Not using engine.train() since the original script isn't updated
results = train(model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=5)

 20%|██        | 1/5 [00:01<00:07,  1.91s/it]

Epoch: 1 | train_loss: 0.2990 | train_acc: 0.9381 | test_loss: 0.4921 | test_acc: 0.8229


 40%|████      | 2/5 [00:03<00:05,  1.95s/it]

Epoch: 2 | train_loss: 0.2319 | train_acc: 0.9713 | test_loss: 0.5957 | test_acc: 0.8021


 60%|██████    | 3/5 [00:05<00:03,  1.93s/it]

Epoch: 3 | train_loss: 0.2114 | train_acc: 0.9679 | test_loss: 0.3841 | test_acc: 0.8698


 80%|████████  | 4/5 [00:07<00:01,  1.92s/it]

Epoch: 4 | train_loss: 0.1943 | train_acc: 0.9728 | test_loss: 0.5046 | test_acc: 0.8333


100%|██████████| 5/5 [00:09<00:00,  1.93s/it]

Epoch: 5 | train_loss: 0.1818 | train_acc: 0.9649 | test_loss: 0.4839 | test_acc: 0.8333


## Create a helper function to build `SummaryWriter()`(s)

In [71]:
from torchvision.models import efficientnet_b4
effnetb4_in_features = 1792

In [72]:
effnetb4 = efficientnet_b4(pretrained=True).to(device)
effnetb4

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [73]:
for param in effnetb4.features.parameters():
    param.requires_grad = False


In [74]:
effnetb4.classifier = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(in_features=1792, out_features=len(class_names))
).to(device)

In [75]:
def create_writer(experiment_name, model_name, extra=None):
    assert experiment_name, "experiment_name can't be None"
    assert model_name, "model_name can't be None"

    if extra:
        log_dir = os.path.join(experiment_name, model_name, extra)
    else:
        log_dir = os.path.join(experiment_name, model_name)
    print(f"Created SummaryWriter and saving to {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [76]:
def train(
    model,
    train_dataloader,
    test_dataloader,
    optimizer,
    loss_fn=nn.CrossEntropyLoss(),
    epochs=5,
    writer=None
):

    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
        )
        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn
        )

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss", 
                            tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                            global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                            tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc}, 
                            global_step=epoch)
        else:
            pass
    
    if writer:
        # Close the writer
        writer.close()

    return results

## Setup a series of modelling experiments

In [77]:
optimizer = torch.optim.Adam(params=effnetb4.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # label smoothing from here: https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/

In [79]:
results = train(model=effnetb4,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=5,
    writer=create_writer(experiment_name="no_data_augmentation", model_name="effnetb4", extra="label_smoothing")
)

Created SummaryWriter and saving to no_data_augmentation/effnetb4/label_smoothing...


 20%|██        | 1/5 [00:02<00:11,  2.81s/it]

Epoch: 1 | train_loss: 1.2164 | train_acc: 0.8461 | test_loss: 1.3528 | test_acc: 0.7344


 40%|████      | 2/5 [00:05<00:08,  2.79s/it]

Epoch: 2 | train_loss: 1.1285 | train_acc: 0.8663 | test_loss: 1.2134 | test_acc: 0.8594


...

Epoch: 3 | train_loss: 1.0807 | train_acc: 0.8741 | test_loss: 1.1759 | test_acc: 0.8542


 80%|████████  | 4/5 [00:11<00:02,  2.76s/it]

Epoch: 4 | train_loss: 1.0105 | train_acc: 0.8890 | test_loss: 1.1328 | test_acc: 0.8229


100%|██████████| 5/5 [00:13<00:00,  2.77s/it]

Epoch: 5 | train_loss: 0.9681 | train_acc: 0.8890 | test_loss: 1.0863 | test_acc: 0.8542


## TOD0: Create differnet series of experiments
* Each folder could be a different experiment type?
    * e.g. `runs/model_name/data_aug_strategy/...`
* Or each folder could contain a certain model with different parameters
    * e.g. `runs/model_name/1234_aug_strategy/...`
* Or each folder could contain the experiment style?
    * e.g. `runs/data_augmentation/model_name_strategy/...`
    * and `runs/no_augmentation/model_name/...`